# Scrapy Fundamentals — 45 Worked Examples (15 per section)

Sections:
1) **Scrapy architecture (Spider, Item, Pipeline)**
2) **Creating a basic spider**
3) **CSS/XPath selectors**

Each example includes:
- **Markdown**: What the function/feature is about
- **Code**: Minimal, copy-ready snippet
- **Explanation**: How it works in detail
- **Optimization options**: Tips for performance, reliability, and clarity

> Note: These cells are designed for a real Scrapy environment (install with `pip install scrapy`).
> Networking is disabled in this environment, but the snippets will run in your local project.


## Section 1 — Scrapy architecture (Spider, Item, Pipeline) (15 Examples)

### Project scaffold & anatomy
Create a project and understand directories: settings, pipelines, spiders.

In [ ]:
# Terminal (run outside Python)
# pip install scrapy
# scrapy startproject myproj
# cd myproj && tree -L 2


**Explanation**
Scrapy projects have a top-level `scrapy.cfg` and a package (e.g., `myproj/`) that contains `settings.py`, `pipelines.py`, `middlewares.py`, and a `spiders/` folder. You write spider classes in `spiders/`.

**Optimization options**
- Keep projects versioned; separate common code (pipelines, loaders) into modules for reuse.

### Defining Items with scrapy.Item
Define a structured container for scraped data.

In [ ]:
# items.py
import scrapy

class QuoteItem(scrapy.Item):
    text = scrapy.Field()
    author = scrapy.Field()
    tags = scrapy.Field()


**Explanation**
`scrapy.Item` behaves like a dict but documents expected fields. Use it to centralize the schema of your output.

**Optimization options**
- Prefer Items when teams collaborate—schemas are explicit; use `dataclasses`/Pydantic for validation-heavy workflows.

### Alternative: dataclasses for Items
Use Python dataclasses instead of Item for type hints/validation.

In [ ]:
# items_dc.py
from dataclasses import dataclass
from typing import List

@dataclass
class QuoteDC:
    text: str
    author: str
    tags: List[str]


**Explanation**
Dataclasses give type hints and nice reprs; you can still export them via Feed exports by yielding dicts (e.g., `asdict(dc)`).

**Optimization options**
- Use dataclasses when you want IDE type checking; convert to dicts in pipelines/exports.

### Spider skeleton
Minimal spider: `name`, `start_urls`, and `parse` callback.

In [ ]:
# spiders/minimal_spider.py
import scrapy

class MinimalSpider(scrapy.Spider):
    name = "minimal"
    start_urls = ["https://example.com/"]

    def parse(self, response):
        yield {"title": response.css("title::text").get(), "url": response.url}


**Explanation**
Spiders produce Requests and Items. With `start_urls`, Scrapy auto-generates initial `Request`s to each URL and calls `parse`.

**Optimization options**
- For dynamic seeds, implement `start_requests()` to generate Requests with headers/cookies.

### start_requests vs start_urls
Generate initial requests manually (e.g., POST, custom headers).

In [ ]:
# spiders/startreq_spider.py
import scrapy

class StartReqSpider(scrapy.Spider):
    name = "startreq"
    def start_requests(self):
        yield scrapy.Request("https://example.com/", headers={"User-Agent":"MyBot/1.0"}, callback=self.parse)

    def parse(self, response):
        yield {"title": response.css("title::text").get()}


**Explanation**
`start_requests()` gives full control over method, headers, cookies, and metadata for the seed requests.

**Optimization options**
- Prefer `start_urls` for simple GETs; switch to `start_requests()` when you need control or auth tokens.

### Request meta & cb_kwargs
Pass context across requests via `meta` or typed args via `cb_kwargs`.

In [ ]:
# spiders/meta_kwargs.py
import scrapy

class MetaKwargsSpider(scrapy.Spider):
    name = "meta_kwargs"
    start_urls = ["https://example.com/"]
    def parse(self, response):
        for href in response.css("a::attr(href)").getall():
            yield response.follow(href, callback=self.parse_page, cb_kwargs={"from_url": response.url}, meta={"depth":1})
    def parse_page(self, response, from_url):
        yield {"url": response.url, "from": from_url, "depth": response.meta.get("depth")}


**Explanation**
`meta` can carry arbitrary (picklable) data and is accessible via `response.meta`. `cb_kwargs` passes named arguments directly into the callback signature.

**Optimization options**
- Use `cb_kwargs` for simple typed parameters; reserve `meta` for internal flags (depth, dont_redirect, proxies).

### Errbacks & failure handling
Handle network errors and exceptions per-request.

In [ ]:
# spiders/errback_spider.py
import scrapy

class ErrbackSpider(scrapy.Spider):
    name = "errback"
    start_urls = ["https://example.com/404"]
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, callback=self.parse, errback=self.on_error, dont_filter=True)
    def parse(self, response):
        yield {"status": response.status, "url": response.url}
    def on_error(self, failure):
        self.logger.warning("Failed: %r", failure.request.url)


**Explanation**
If a request fails (DNS, timeout, HTTP error when `handle_httpstatus_list` not used), the `errback` runs with a `Failure` object. Log and possibly retry or route to backup URLs.

**Optimization options**
- Add RetryMiddleware settings for global policy; use idempotent callbacks to safely retry.

### Pipelines: validation & persistence
Post-process Items (dedupe, clean, save to CSV/DB).

In [ ]:
# pipelines.py
import csv, scrapy

class SaveCsvPipeline:
    def open_spider(self, spider):
        self.seen=set()
        self.f=open('items.csv','w',newline='',encoding='utf-8')
        self.w=csv.DictWriter(self.f, fieldnames=['text','author','tags'])
        self.w.writeheader()
    def close_spider(self, spider):
        self.f.close()
    def process_item(self, item, spider):
        key=(item.get('text'), item.get('author'))
        if key in self.seen:
            raise scrapy.exceptions.DropItem("Duplicate")
        self.seen.add(key); self.w.writerow(item); return item

# settings.py
ITEM_PIPELINES={'myproj.pipelines.SaveCsvPipeline':300}


**Explanation**
Pipelines receive each scraped item in order of priority (lower number first). Use them to validate fields, drop duplicates, normalize text, and write to storage.

**Optimization options**
- Avoid heavy DB I/O in `process_item`—batch writes or use async drivers; validate early to fail fast.

### Item Loaders with processors
Normalize fields at extraction time using processors.

In [ ]:
# loaders.py
from scrapy.loader import ItemLoader
from itemloaders.processors import MapCompose, TakeFirst, Join
import re

def clean_text(s): return re.sub(r"\s+", " ", s or "").strip()

class QuoteLoader(ItemLoader):
    default_output_processor = TakeFirst()
    text_in = MapCompose(str.strip, clean_text)
    tags_out = Join(",")

# Usage in spider:
# l = QuoteLoader(item=QuoteItem(), response=response)
# l.add_css("text", "span.text::text")
# l.add_css("author","small.author::text")
# l.add_css("tags","div.tags a.tag::text")
# yield l.load_item()


**Explanation**
Loaders let you declaratively clean data (strip, regex, join) before yielding final items, keeping spiders concise.

**Optimization options**
- Do heavy normalization in pipelines, light normalization in loaders; keep processors pure & testable.

### Downloader middlewares (headers, proxies, retries)
Globally tweak requests/responses (UA, cookies, retry, proxy).

In [ ]:
# settings.py (snippets)
DEFAULT_REQUEST_HEADERS = {"User-Agent": "MyScraper/1.0 (+http://example.com/bot)"}
RETRY_ENABLED = True
RETRY_TIMES = 3
# PROXY via env or custom middleware
# DOWNLOADER_MIDDLEWARES = {'myproj.middlewares.ProxyMiddleware': 750}


**Explanation**
Downloader middlewares run around the Downloader stage. Use built-ins for retries and cookies; add custom middleware for proxy rotation or header injection.

**Optimization options**
- Keep middleware small and orthogonal; prefer config over code where possible.

### AutoThrottle for polite crawling
Dynamically adapts concurrency & delay to server latency.

In [ ]:
# settings.py
AUTOTHROTTLE_ENABLED = True
AUTOTHROTTLE_START_DELAY = 0.5
AUTOTHROTTLE_MAX_DELAY = 10.0
AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0


**Explanation**
AutoThrottle measures latency and adjusts delays to avoid overwhelming servers while keeping throughput steady.

**Optimization options**
- Combine with `CONCURRENT_REQUESTS_PER_DOMAIN` and `DOWNLOAD_DELAY` for stable behavior.

### HTTP cache for dev speed
Cache responses locally during development.

In [ ]:
# settings.py
HTTPCACHE_ENABLED = True
HTTPCACHE_EXPIRATION_SECS = 0
HTTPCACHE_DIR = 'httpcache'


**Explanation**
Caching speeds up iteration, reduces load on the site, and lets you debug selectors offline. Disable for production runs.

**Optimization options**
- Remember to bust cache when HTML structure changes or auth/session differs.

### Feed exports (JSON/CSV/Parquet)
Use FEEDS to declare outputs without custom code.

In [ ]:
# settings.py
FEEDS = {
    "items.json": {"format":"json", "overwrite": True, "indent":2},
    "items.csv": {"format":"csv", "overwrite": True, "encoding":"utf-8"},
}


**Explanation**
FEEDS eliminates manual file handling in pipelines for simple exports—Scrapy handles serialization and rotation.

**Optimization options**
- Use timestamped paths like `items-%(time)s.json` for reproducible runs.

### Scheduler & dupefilter
Request queueing & duplicate filtering via fingerprints.

In [ ]:
# usage in spider
yield scrapy.Request(url, callback=self.parse)             # deduped normally
yield scrapy.Request(url, callback=self.parse, dont_filter=True)  # force re-request


**Explanation**
By default, Scrapy avoids re-requesting the same URL (fingerprint). Use `dont_filter=True` to bypass (e.g., different payloads) or include non-URL params in the fingerprint.

**Optimization options**
- When re-running a job, consider `-s JOBDIR=state/` to persist the queue and fingerprints.

### Signals & extensions
Hook into crawl lifecycle (open/close spider, stats).

In [ ]:
# extensions.py
from scrapy import signals

class StatsPrinter:
    @classmethod
    def from_crawler(cls, crawler):
        ext = cls()
        crawler.signals.connect(ext.spider_opened, signal=signals.spider_opened)
        crawler.signals.connect(ext.spider_closed, signal=signals.spider_closed)
        return ext
    def spider_opened(self, spider):
        spider.logger.info("Opened spider: %s", spider.name)
    def spider_closed(self, spider, reason):
        spider.logger.info("Closed spider: %s (%s)", spider.name, reason)

# settings.py
EXTENSIONS={'myproj.extensions.StatsPrinter': 500}


**Explanation**
Signals enable cross-cutting concerns (metrics, alerts). Extensions are initialized once and can access the crawler object.

**Optimization options**
- Keep extensions generic and configurable via settings to reuse across projects.

## Section 2 — Creating a basic spider (15 Examples)

### Quotes spider: parse list page
Scrape quotes, authors, and tags from the homepage.

In [ ]:
# spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    allowed_domains = ["quotes.toscrape.com"]
    start_urls = ["https://quotes.toscrape.com/"]

    def parse(self, response):
        for q in response.css("div.quote"):
            yield {
                "text": q.css("span.text::text").get(),
                "author": q.css("small.author::text").get(),
                "tags": q.css("div.tags a.tag::text").getall(),
            }


**Explanation**
Iterate `div.quote` blocks; for each block, use relative CSS selectors to extract fields. Yield dicts as items for the FEEDS exporter.

**Optimization options**
- Narrow scopes: select container then child selectors to avoid bleed-through.

### Pagination with response.follow
Follow the 'next' link to scrape multiple pages.

In [ ]:
def parse(self, response):
    for q in response.css("div.quote"):
        yield {
            "text": q.css("span.text::text").get(),
            "author": q.css("small.author::text").get(),
            "tags": q.css("div.tags a.tag::text").getall(),
        }
    next_page = response.css("li.next a::attr(href)").get()
    if next_page:
        yield response.follow(next_page, callback=self.parse)


**Explanation**
`response.follow` resolves relative URLs against the current response URL, preserving headers/cookies automatically.

**Optimization options**
- Always check `next_page` is not None; use `cb_kwargs` to carry state when needed.

### Detail page crawl with meta/cb_kwargs
Open author pages to fetch extra info, then merge back.

In [ ]:
def parse(self, response):
    for q in response.css("div.quote"):
        author_url = q.css("span a::attr(href)").get()
        item = {
            "text": q.css("span.text::text").get(),
            "author": q.css("small.author::text").get(),
            "tags": q.css("div.tags a.tag::text").getall(),
        }
        if author_url:
            yield response.follow(author_url,
                                  callback=self.parse_author,
                                  cb_kwargs={"item": item})
        else:
            yield item

def parse_author(self, response, item):
    item["author_born"] = response.css("span.author-born-date::text").get()
    item["author_loc"] = response.css("span.author-born-location::text").get()
    yield item


**Explanation**
Carry a partially filled item to the detail page using `cb_kwargs`. After enriching, yield the item. This avoids temporary storage or global state.

**Optimization options**
- Avoid deep nesting; consider separate spiders for complex multi-hop crawls.

### Login example with FormRequest
Simulate login before scraping protected pages.

In [ ]:
import scrapy
from scrapy.http import FormRequest

class LoginSpider(scrapy.Spider):
    name = "login_example"
    start_urls = ["https://example.com/login"]
    def parse(self, response):
        return FormRequest.from_response(
            response,
            formid="loginForm",
            formdata={"username":"u", "password":"p"},
            callback=self.after_login
        )
    def after_login(self, response):
        if "invalid" in response.text.lower():
            self.logger.error("Login failed")
            return
        yield response.follow("/dashboard", callback=self.parse_dashboard)


**Explanation**
`FormRequest.from_response` parses the login form, preserves cookies, and submits credentials. Post-login, you can follow app pages with the authenticated session.

**Optimization options**
- Use `dont_filter=True` on post-login redirects if needed; store secrets in env vars, not code.

### JSON API spider
Request a JSON endpoint and parse structured data.

In [ ]:
class ApiSpider(scrapy.Spider):
    name = "api"
    start_urls = ["https://api.example.com/items?page=1"]
    def parse(self, response):
        data = response.json()
        for it in data["items"]:
            yield {"id": it["id"], "name": it["name"]}
        if data.get("next"):
            yield response.follow(data["next"], callback=self.parse)


**Explanation**
Use `response.json()` to parse JSON directly when content-type is JSON. Paginate via the `next` link/key if provided.

**Optimization options**
- Cap pagination and validate schema—APIs may change silently.

### Throttle and politeness
Control concurrency/delay; enable AutoThrottle.

In [ ]:
class PoliteSpider(scrapy.Spider):
    name = "polite"
    custom_settings = {
        "ROBOTSTXT_OBEY": True,
        "DOWNLOAD_DELAY": 0.5,
        "AUTOTHROTTLE_ENABLED": True,
        "CONCURRENT_REQUESTS_PER_DOMAIN": 4,
    }


**Explanation**
Per-spider `custom_settings` lets you tune politeness without touching global settings.

**Optimization options**
- Start conservative, then raise concurrency after measuring server latency.

### CrawlSpider + Rule + LinkExtractor
Rule-driven following of URL patterns.

In [ ]:
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

class BlogSpider(CrawlSpider):
    name = "blog"
    allowed_domains = ["example.com"]
    start_urls = ["https://example.com/blog/"]
    rules = (
        Rule(LinkExtractor(allow=r"/blog/page/\d+"), follow=True),
        Rule(LinkExtractor(allow=r"/blog/\d+/\d+/[\w-]+"), callback="parse_post"),
    )
    def parse_post(self, response):
        yield {"title": response.css("h1::text").get(), "url": response.url}


**Explanation**
`CrawlSpider` automates link-following via rules. `LinkExtractor` finds links; `Rule` controls which to follow and which to parse.

**Optimization options**
- Test rules in `scrapy shell` to ensure patterns match exactly; avoid overly broad regexes.

### Headers & cookies per request
Customize requests that need specific headers or cookies.

In [ ]:
def parse(self, response):
    for href in response.css("a.special::attr(href)").getall():
        yield response.follow(href, callback=self.parse_page,
                              headers={"Accept-Language":"en-US"},
                              cookies={"mkt":"us"})


**Explanation**
Sometimes APIs or geo-targeted sites rely on headers/cookies. You can override them per-request.

**Optimization options**
- Centralize defaults in settings/middlewares; override only when needed.

### Custom settings per spider
Tweak exporters, pipelines, concurrency for a single spider.

In [ ]:
class TunedSpider(scrapy.Spider):
    name = "tuned"
    custom_settings = {
        "FEEDS": {"items.json":{"format":"json","overwrite":True,"indent":2}},
        "ITEM_PIPELINES": {"myproj.pipelines.SaveCsvPipeline": 300},
        "DOWNLOAD_DELAY": 0.25,
    }


**Explanation**
`custom_settings` merges with global settings at spider load. Useful for experiments or special targets.

**Optimization options**
- Avoid drifting configurations—document why a spider diverges from defaults.

### Simple CSV via FEEDS
Export results to CSV without writing code.

In [ ]:
# Run from terminal:
# scrapy crawl quotes -O quotes.csv   # overwrite (since Scrapy 2.1+)


**Explanation**
`-O` overwrites the output; `-o` appends/updates. FEEDS handles serialization.

**Optimization options**
- Prefer FEEDS for simple use-cases; graduate to pipelines for complex formats or DB writes.

### Files/Images pipelines (reference)
Download images/files referenced by items.

In [ ]:
# settings.py (snippets)
ITEM_PIPELINES = {
  'scrapy.pipelines.images.ImagesPipeline': 1,
}
IMAGES_STORE = 'images'
# Item fields must include image URLs, e.g., 'image_urls': ['http://...']


**Explanation**
Built-in media pipelines handle downloading & caching. Items need URL fields with specific names (e.g., `image_urls`).

**Optimization options**
- Use per-domain concurrency caps to avoid hammering media hosts.

### Close spider by limit
Stop after N items/pages to contain runs.

In [ ]:
# settings.py (snippets)
CLOSESPIDER_ITEMCOUNT = 1000
CLOSESPIDER_PAGECOUNT = 500


**Explanation**
CloseSpider extension auto-stops the crawl when thresholds are reached, keeping costs/time bounded.

**Optimization options**
- Use for smoke tests or cron jobs to prevent runaway crawls.

### Persist crawl state with JOBDIR
Resume a crawl preserving queue and dupefilter.

In [ ]:
# terminal
# scrapy crawl quotes -s JOBDIR=state/quotes_run


**Explanation**
JOBDIR stores scheduler state, fingerprints, and requests so you can resume later without redoing work.

**Optimization options**
- Choose separate directories per spider/target to avoid collisions.

### Run spiders programmatically
Use `CrawlerProcess` to run spiders from a script.

In [ ]:
# run_spider.py
import scrapy
from scrapy.crawler import CrawlerProcess

class HelloSpider(scrapy.Spider):
    name="hello"
    start_urls=["https://example.com/"]
    def parse(self, response):
        yield {"title": response.css("title::text").get()}

process = CrawlerProcess(settings={
    "FEEDS":{"items.json":{"format":"json","overwrite":True}}
})
process.crawl(HelloSpider)
process.start()


**Explanation**
`CrawlerProcess` embeds Scrapy into a Python script. Handy for CI or orchestrating multiple spiders with different settings.

**Optimization options**
- Avoid blocking scripts inside notebooks; prefer separate runner scripts.

## Section 3 — CSS/XPath selectors (15 Examples)

### CSS basics: text
Select text nodes with `::text`.

In [ ]:
# In parse()
title = response.css("title::text").get()
all_quotes = response.css("div.quote span.text::text").getall()


**Explanation**
`::text` accesses text node contents. `.get()` returns the first match; `.getall()` returns a list.

**Optimization options**
- Chain selectors from a container (e.g., `div.quote`) to reduce accidental matches.

### CSS attributes via ::attr(name)
Extract attributes such as `href` or `src`.

In [ ]:
links = response.css("a::attr(href)").getall()
imgs = response.css("img::attr(src)").getall()


**Explanation**
`::attr(name)` returns the attribute value(s) of selected nodes.

**Optimization options**
- Join relative URLs with `response.urljoin` or just use `response.follow` for links.

### Scoped CSS selection
Iterate over blocks, then select children relative to each block.

In [ ]:
for block in response.css("div.quote"):
    text = block.css("span.text::text").get()
    author = block.css("small.author::text").get()
    tags = block.css("div.tags a.tag::text").getall()
    yield {"text":text, "author":author, "tags":tags}


**Explanation**
Scoping prevents mixing fields from different blocks and is faster than repeated global queries.

**Optimization options**
- Cache sub-selections in variables if reused to avoid recomputation.

### Regex extraction with .re()
Use regex against the HTML/text of the selection.

In [ ]:
ids = response.css("div.post").re(r'id-(\d+)')  # ['123','456']


**Explanation**
`.re(pattern)` applies a regex to the HTML of the selected nodes; it returns strings.

**Optimization options**
- Prefer structured selectors for robust scraping; use regex as a last mile.

### XPath basics: text
Select node text using XPath.

In [ ]:
title = response.xpath("//title/text()").get()
quotes = response.xpath("//div[@class='quote']//span[@class='text']/text()").getall()


**Explanation**
XPath is powerful for structural queries; `//` is a deep search, `@` accesses attributes.

**Optimization options**
- Use explicit paths instead of chained `//` to avoid surprises.

### XPath attributes
Extract attributes with `/@attr`.

In [ ]:
hrefs = response.xpath("//a/@href").getall()


**Explanation**
XPath uses `/@name` to access attributes of the current node set.

**Optimization options**
- Combine with predicates to filter links: `//a[contains(@href,'page')]`.

### XPath contains/starts-with
Filter nodes by substring or prefix.

In [ ]:
tags = response.xpath("//a[contains(@class,'tag')]/text()").getall()
imgs = response.xpath("//img[starts-with(@src,'/static/')]/@src").getall()


**Explanation**
`contains()` and `starts-with()` are robust for matching classes/URLs that include extra tokens.

**Optimization options**
- Prefer token-aware matching for classes (split by space) to avoid partial hits.

### XPath normalize-space
Trim whitespace and collapse internal runs.

In [ ]:
heading = response.xpath("normalize-space(//h1)").get()


**Explanation**
`normalize-space()` removes leading/trailing whitespace and collapses consecutive spaces to single.

**Optimization options**
- Use on headings or messy text nodes to clean up without extra Python code.

### XPath position()
Pick nth elements or ranges with `position()`.

In [ ]:
top3 = response.xpath("(//div[@class='quote'])[position()<=3]//span[@class='text']/text()").getall()


**Explanation**
`position()` helps select top-N or every nth element without post-filtering in Python.

**Optimization options**
- Do not overuse positional selectors on dynamic lists; prefer semantic filters.

### Table extraction (XPath)
Select table rows and cells structurally.

In [ ]:
rows = []
for tr in response.xpath("//table[@id='tbl']/tbody/tr"):
    cells = tr.xpath("./td//text()").getall()
    rows.append([x.strip() for x in cells if x.strip()])


**Explanation**
Tables often have nested tags; use `//text()` within each `td` and strip in Python.

**Optimization options**
- Prefer CSS `table#tbl tbody > tr` when HTML is simple; XPath shines on complex nesting.

### Combine CSS then XPath
Start with CSS and refine with XPath relative to selection.

In [ ]:
for post in response.css("article.post"):
    date = post.xpath(".//time/@datetime").get()
    title = post.css("h2 a::text").get()
    yield {"title": title, "date": date}


**Explanation**
Selectors are chainable; the dot `.` ensures XPath is relative to the current node.

**Optimization options**
- Mixing CSS and XPath can be pragmatic—choose the clearer expression.

### Relative vs absolute XPath
Use `.` to stay within a scoped node.

In [ ]:
.asserted = []
for card in response.xpath("//div[@class='card']"):
    link = card.xpath(".//a/@href").get()  # relative to card
    # Without the dot, it would search the full document each time


**Explanation**
Relative XPath reuses the current context node, improving performance and correctness.

**Optimization options**
- Always include `.` for child selections inside loops.

### Cleaning and joining text
Collect multiple text nodes and join them cleanly.

In [ ]:
raw = response.css("div.article p::text").getall()
text = " ".join([t.strip() for t in raw if t.strip()])


**Explanation**
HTML often splits text across nodes—gather, strip, and join to reconstruct paragraphs.

**Optimization options**
- Prefer `normalize-space(string(//node))` in XPath for quick inline cleanup.

### get vs getall defaults
Use sensible defaults to avoid `None` errors.

In [ ]:
title = response.css("title::text").get(default="(no title)")
tags = response.css("a.tag::text").getall() or []


**Explanation**
`.get(default=...)` prevents `None` when a selector misses; `.getall()` returns `[]` when empty.

**Optimization options**
- Validate counts (e.g., assert at least one tag) to catch selector drift early.